In [2]:
import pandas as pd

import numpy as np
import sqlite3
from simsopt.mhd import Vmec
from simsopt.mhd import QuasisymmetryRatioResidual
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping

ModuleNotFoundError: No module named 'seaborn'

# Loading Data

In [ ]:
conn = sqlite3.connect('../data/nfp2/nfp2.db')  # Adjust the path to your database file

# Step 2 & 3: Query the database and load the data into a pandas DataFrame
query = "SELECT * FROM stellarators"  # Adjust your query as needed
data_df = pd.read_sql_query(query, conn)

: 

# Creating scaled and not scaled variables

In [ ]:
data_df_clean = data_df.dropna(subset=['quasisymmetry'])
X = data_df_clean[['rbc_1_0', 'rbc_m1_1', 'rbc_0_1', 'rbc_1_1','zbs_1_0', 'zbs_m1_1', 'zbs_0_1', 'zbs_1_1']] 
Y = data_df_clean[['quasisymmetry', 'quasiisodynamic', 'rotational_transform', 'inverse_aspect_ratio', 'mean_local_magnetic_shear', 'vacuum_magnetic_well', 'maximum_elongation', 'mirror_ratio']] 

X_train, X_val_test, Y_train, Y_val_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_validation, X_test, Y_validation, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5, random_state=42)


y_to_train = Y_train['quasisymmetry']
y_to_test = Y_test['quasisymmetry']
y_to_validation = Y_validation['quasisymmetry']

xscaler = StandardScaler()
yscaler = StandardScaler()  # Using MinMaxScaler for Y as well

# Assuming y_to_train and y_to_test are Series and need reshaping to fit the scaler
Y_train_reshaped = y_to_train.values.reshape(-1, 1)
Y_test_reshaped = y_to_test.values.reshape(-1, 1)
Y_val_reshaped = y_to_validation.values.reshape(-1, 1)

# Fit and transform the training data
Y_train_scaled = yscaler.fit_transform(Y_train_reshaped)
X_train_scaled = xscaler.fit_transform(X_train)

# Transform the testing data (no fitting!)
Y_test_scaled = yscaler.transform(Y_test_reshaped)
X_test_scaled = xscaler.transform(X_test)

X_validation_scaled = xscaler.transform(X_validation)
Y_validation_scaled = yscaler.transform(y_to_validation.values.reshape(-1, 1))

Y_train_scaled_1d = Y_train_scaled.ravel()
Y_validation_scaled_1d = Y_validation_scaled.ravel()


: 

# Testing HistGradientBoostingRegressor 

* Not scaled -> 1st Cell

In [ ]:
# Define a grid of hyperparameters to search
param_grid = {
    'max_iter': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, None],
}

model = HistGradientBoostingRegressor()

# Setup the GridSearchCV 
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error') # MAE

# Perform the grid search with cross-validation
grid_search.fit(X_train, y_to_train)

# Best model after grid search
best_model = grid_search.best_estimator_

# Predict on the testing set
Y_pred = best_model.predict(X_test)

print(Y_pred.shape, y_to_test.shape)

# Calculate MSE for the test set
mse = mean_squared_error(y_to_test, Y_pred)

print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Test MSE: {mse}")

: 

* Scaled -> 2nd Cell 

In [ ]:
# Doing the same but with scaling
grid_search.fit(X_train_scaled, Y_train_scaled_1d)

# Best model after grid search
best_model = grid_search.best_estimator_

# Predict on the testing set
Y_pred_scaled = best_model.predict(X_test_scaled)


# Calculate MSE for the test set
mse = mean_squared_error(Y_test_scaled, Y_pred_scaled)

print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Test MSE: {mse}")

: 

## Testing LightGBMLSS

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train_scaled, label=Y_train_scaled.ravel())
val_data = lgb.Dataset(X_validation_scaled, label=Y_validation_scaled.ravel())

# Define parameters for LightGBM
params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',   # Use Gradient Boosting Decision Tree
    'metric': 'rmse',           # Mean Squared Error as the evaluation metric
    'verbosity': 0             # Set verbosity to 0 to suppress unnecessary messages
}

# Prepare an empty dictionary to store evaluation results
evals_result = {}

# Train the LightGBM model with record_evaluation callback
num_rounds = 1000  # Number of boosting rounds
early_stopping_rounds = 500  # Early stopping rounds to prevent overfitting
valid_sets = [train_data, val_data]
model = lgb.train(params, train_data, num_boost_round=num_rounds, valid_sets=valid_sets, 
                  callbacks=[lgb.record_evaluation(evals_result)])

print(lgb.record_evaluation(evals_result))
# Predict on the test set
Y_pred_scaled = model.predict(X_test_scaled, num_iteration=model.best_iteration)

# Inverse scaling on the predicted values if necessary
# y_pred = yscaler.inverse_transform(Y_pred_scaled.reshape(-1, 1))

# Calculate Mean Squared Error
mse = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("Mean Squared Error:", mse)


# Plot metrics using the recorded evaluation results
lgb.plot_metric(evals_result)



: 

## Testing  XGBoostLSS

In [ ]:
# Define the XGBoost regressor
model = XGBRegressor(
    objective='reg:squarederror',  # Specify squared error as the objective for regression
    tree_method='hist',            # Use 'hist' method for faster training
    eval_metric='rmse',            # Root Mean Squared Error as the evaluation metric
    booster='gbtree',              # Use tree-based model
    verbosity=0,                    # Set verbosity to 0 to suppress unnecessary messages
    device = 'cuda'
)

# Train the model with early stopping
model.fit(X_train_scaled, Y_train_scaled,
    eval_set=[(X_train_scaled, Y_train_scaled), (X_validation_scaled, Y_validation_scaled)], 
    early_stopping_rounds=100,     # Stop if no improvement in 50 rounds
    verbose=False,                                  
)

# Predict on the test set
y_pred_scaled = model.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(Y_test_scaled, y_pred_scaled)
print("Mean Squared Error:", mse)

results = model.evals_result()
print(results['validation_1'])
# plot learning curves
plt.plot(results['validation_0']['rmse'], label='Train RMSE')
plt.plot(results['validation_1']['rmse'], label='Validation RMSE')
plt.legend()
plt.show()



: 

# Using TSNE to try to visualize the problem

In [ ]:
from sklearn.manifold import TSNE

# Apply t-SNE to the scaled training features
truncated_df = data_df_clean.truncate(before = 100, after = 10000)

features = truncated_df[['rbc_1_0', 'rbc_m1_1', 'rbc_0_1', 'rbc_1_1', 'zbs_1_0', 'zbs_m1_1', 'zbs_0_1', 'zbs_1_1']]
quasisymmetry = truncated_df['quasisymmetry']  # This is the 9th dimension used for coloring

# Apply t-SNE to the features to reduce to 2D for visualization
tsne = TSNE(n_components=2, random_state=9, perplexity=50)
features_reduced = tsne.fit_transform(features)

# Plotting
plt.figure(figsize=(10, 8))
scatter = plt.scatter(features_reduced[:, 0], features_reduced[:, 1], c=quasisymmetry, cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Quasisymmetry Value')
plt.title('t-SNE plot of the first 8 dimensions colored by Quasisymmetry')
plt.xlabel('t-SNE Feature 1')
plt.ylabel('t-SNE Feature 2')
plt.show()

: 

In [ ]:
# Re-importing necessary libraries after reset
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

truncated_df = data_df_clean.truncate(before = 100, after = 10000)

features_dummy = truncated_df[['rbc_1_0', 'rbc_m1_1', 'rbc_0_1', 'rbc_1_1', 'zbs_1_0', 'zbs_m1_1', 'zbs_0_1', 'zbs_1_1']]
quasisymmetry_dummy = truncated_df['quasisymmetry']  # This is the 9th dimension used for coloring

# Apply t-SNE to the features to reduce to 3D for visualization
tsne_dummy = TSNE(n_components=3, random_state=9, perplexity=50)
features_reduced_dummy = tsne_dummy.fit_transform(features_dummy)

# 3D plotting
fig_dummy = plt.figure(figsize=(10, 8))
ax_dummy = fig_dummy.add_subplot(111, projection='3d')

# Scatter plot for 3D t-SNE components
scatter_dummy = ax_dummy.scatter(features_reduced_dummy[:, 0], features_reduced_dummy[:, 1], features_reduced_dummy[:, 2], c=quasisymmetry_dummy, cmap='viridis', alpha=0.6)
fig_dummy.colorbar(scatter_dummy, label='Quasisymmetry Value')

plt.title('3D t-SNE plot of the first 8 dimensions colored by Quasisymmetry (Dummy Data)')
ax_dummy.set_xlabel('t-SNE Feature 1')
ax_dummy.set_ylabel('t-SNE Feature 2')
ax_dummy.set_zlabel('t-SNE Feature 3')
plt.show()


: 

: 